# Симуляция распространения болезни. Работа с NetworkX

https://www.kaggle.com/datasets/vikalpdongre/us-flights-data-2008

# Симуляция

In [102]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

import networkx as nx
from collections import defaultdict
from tqdm.auto import tqdm
from datetime import datetime, timedelta

In [103]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
 
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [104]:
link = 'https://drive.google.com/file/d/13WHMbPp3E8Wx3zM9bRn_nIvJ-DuhdyGO/view?usp=share_link'
 
# to get the id part of the file
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('2008.csv') 
 
df = pd.read_csv('2008.csv')
df.head(6).T

,0,1,2,3,4,5
Year,2008,2008,2008,2008,2008,2008
Month,1,1,1,1,1,1
DayofMonth,3,3,3,3,3,3
DayOfWeek,4,4,4,4,4,4
DepTime,2003.0,754.0,628.0,926.0,1829.0,1940.0
CRSDepTime,1955,735,620,930,1755,1915
ArrTime,2211.0,1002.0,804.0,1054.0,1959.0,2121.0
CRSArrTime,2225,1000,750,1100,1925,2110
UniqueCarrier,WN,WN,WN,WN,WN,WN
FlightNum,335,3231,448,1746,3920,378


In [105]:
# df = pd.read_csv('2008.csv')
# df.head(6).T

In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7009728 entries, 0 to 7009727
Data columns (total 29 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Year               int64  
 1   Month              int64  
 2   DayofMonth         int64  
 3   DayOfWeek          int64  
 4   DepTime            float64
 5   CRSDepTime         int64  
 6   ArrTime            float64
 7   CRSArrTime         int64  
 8   UniqueCarrier      object 
 9   FlightNum          int64  
 10  TailNum            object 
 11  ActualElapsedTime  float64
 12  CRSElapsedTime     float64
 13  AirTime            float64
 14  ArrDelay           float64
 15  DepDelay           float64
 16  Origin             object 
 17  Dest               object 
 18  Distance           int64  
 19  TaxiIn             float64
 20  TaxiOut            float64
 21  Cancelled          int64  
 22  CancellationCode   object 
 23  Diverted           int64  
 24  CarrierDelay       float64
 25  WeatherDelay      

Количество уникальных аэропортов вылета

In [107]:
df.Origin.nunique()

303

Количество уникальных аэропортов прилёта

In [108]:
df.Dest.nunique()

304

Возьмём только нужные нам столбцы

In [109]:
data = df[['Year', 'Month', 'DayofMonth', 'Origin', 'CRSDepTime', 'Dest', 'CRSArrTime']]
data.head(3)

,Year,Month,DayofMonth,Origin,CRSDepTime,Dest,CRSArrTime
0,2008,1,3,IAD,1955,TPA,2225
1,2008,1,3,IAD,735,TPA,1000
2,2008,1,3,IND,620,BWI,750


In [110]:
data.shape

(7009728, 7)

In [111]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7009728 entries, 0 to 7009727
Data columns (total 7 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   Year        int64 
 1   Month       int64 
 2   DayofMonth  int64 
 3   Origin      object
 4   CRSDepTime  int64 
 5   Dest        object
 6   CRSArrTime  int64 
dtypes: int64(5), object(2)
memory usage: 374.4+ MB


In [112]:
def get_time(time):
  """
  Функция преобразования времени к формату 12:34
  """
  str_time = str(time).rjust(4, '0')
  return str_time[:2] + ':' + str_time[2:]
  # return str_time[:2], str_time[2:]

In [113]:
@np.vectorize
def get_datetime(year, month, day, time):
  """
  Функция для преобразования к формату datetime
  """
  h, m = time.split(sep=':')
  result = datetime(year, month, day, int(h), int(m))
  return result

In [114]:
def prepare(data):
  """
  Подготовка датасета к моделированию
  """

  # Заменяем 24:00 на 23:59
  data.loc[data['CRSArrTime'] >= 2400, 'CRSArrTime'] = 2359
  data.loc[data['CRSDepTime'] >= 2400, 'CRSArrTime'] = 2359

  # Создаём колонки с временем вылета и прибытия
  data['Arr_time'] = data['CRSArrTime'].apply(get_time)
  data['Dep_time'] = data['CRSDepTime'].apply(get_time)

  # Создаём колонки datetime
  data['Arrival'] = get_datetime(data['Year'], data['Month'], data['DayofMonth'], data['Arr_time'])
  data['Departure'] = get_datetime(data['Year'], data['Month'], data['DayofMonth'], data['Dep_time'])
  data['Arrival'] = pd.to_datetime(data['Arrival'])
  data['Departure'] = pd.to_datetime(data['Departure'])

  # Оставим только то, что нужно для моделирования
  result = data[['Departure', 'Origin', 'Arrival', 'Dest']]

  # Корректируем время, если прилёт на следующий день
  result.loc[result['Departure'] > result['Arrival'], 'Arrival'] += timedelta(hours=24)

  # Делаем сортировку по времени вылета
  result.sort_values('Departure', inplace=True)

  return result

In [115]:
data = prepare(data)

# Из всего датасета возьмём только один месяц с 2008-01-01 по 2008-01-31
split_date = '2008-02-01'
data = data[data.Arrival < split_date].copy()

In [116]:
data.head(3)

,Departure,Origin,Arrival,Dest
428607,2008-01-01 00:10:00,LAX,2008-01-01 07:37:00,DTW
546846,2008-01-01 00:15:00,SMF,2008-01-01 08:23:00,JFK
488832,2008-01-01 00:25:00,SMF,2008-01-01 05:35:00,DFW


In [117]:
data.tail(3)

,Departure,Origin,Arrival,Dest
518128,2008-01-31 22:55:00,MIA,2008-01-31 23:55:00,TPA
151116,2008-01-31 22:57:00,PHX,2008-01-31 23:53:00,FLG
532349,2008-01-31 23:00:00,OGG,2008-01-31 23:34:00,HNL


In [118]:
data.to_csv('decreased_2008.csv', index=False)

Датасет с ~657 Мб уменьшили до ~27 Мб

In [119]:
data = pd.read_csv('decreased_2008.csv')
data.tail(2)

,Departure,Origin,Arrival,Dest
605367,2008-01-31 22:57:00,PHX,2008-01-31 23:53:00,FLG
605368,2008-01-31 23:00:00,OGG,2008-01-31 23:34:00,HNL


In [120]:
init_datetime = pd.to_datetime({'year': [2008], 'month':[1], 'day': [1], 'hours': [00], 'minutes': [00]})
init_datetime

0   2008-01-01
dtype: datetime64[ns]

In [121]:
# init_datetime = get_datetime([2008], [1], [1], ['00:00'])

In [122]:
data['Arrival'] = pd.to_datetime(data['Arrival'])
data['Departure'] = pd.to_datetime(data['Departure'])

In [123]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 605369 entries, 0 to 605368
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   Departure  605369 non-null  datetime64[ns]
 1   Origin     605369 non-null  object        
 2   Arrival    605369 non-null  datetime64[ns]
 3   Dest       605369 non-null  object        
dtypes: datetime64[ns](2), object(2)
memory usage: 18.5+ MB


In [124]:
def get_simulation(airport, p):
  """
  функция для симуляции одного прохода по всему датасету и распространения инфекции
  """
  infected = [airport]
  results = {airport: init_datetime}

  for row in tqdm(data.itertuples()):
    chance = np.random.uniform()
    if (row.Origin in infected) and (row.Dest not in infected) and chance <= p:  
      infected.append(row.Dest)
      results[row.Dest] = row.Arrival
    else:
      continue
  return results

In [125]:
N = len(data['Origin'].unique())
p = 0.01
airport = 'ABE'
print('Starting airport:', airport)
np.random.seed(42)
res = get_simulation(airport, p)
print(f'Infected in a month: {len(res)}/{N}')

Starting airport: ABE


0it [00:00, ?it/s]

Infected in a month: 233/286


# Как вероятность заражения влияет на скорость распространения?

In [126]:
def run_simulations(airport, prob, n):
    results = pd.DataFrame(columns=['p', 'sim_id', 'airport', 'infected'])
    for p in prob:
        print(f'Simulation for p = {p}')
        for sim in tqdm(range(n)):

            sim_res = pd.DataFrame.from_dict(get_simulation(airport, p), orient='index').reset_index()
            sim_res.columns = ['airport', 'infected']
            sim_res['sim_id'] = sim
            sim_res['p'] = p
            results = pd.concat([results, sim_res]) 

    return results

In [127]:
prob = [0.01, 0.05, 0.1, 0.5, 1]
n = 10
np.random.seed(42)
print('Starting airport:', airport)
results_df = run_simulations(airport, prob, n)
results_df.to_pickle('results_sim.pkl')

Starting airport: ABE
Simulation for p = 0.01


  0%|          | 0/10 [00:00<?, ?it/s]

0it [00:00, ?it/s]

AttributeError: ignored

# Визуализация

In [ ]:
min_time = data['Departure'].min()
print(min_time)

In [ ]:
max_time = data['Arrival'].max()
print(max_time)
# print(type(max_time))

In [ ]:
results_df.head(5)

In [ ]:
min_time = data['Departure'].min()
max_time = data['Arrival'].max()


# Словарь для записи результата
time_sim_res = dict()

# time delta
td = timedelta(hours=12)

# time point
tp = min_time
while tp < max_time:
    infected_pct = results_df[results_df.infected <= tp].groupby(['p', 'sim_id']).size()/N
    infected_pct = infected_pct.reset_index().groupby('p')[0].mean()
    time_sim_res[tp] = infected_pct
    tp += td

In [ ]:
data_viz = pd.DataFrame.from_dict(time_sim_res, orient='index')
data_viz = data_viz.reset_index().melt(id_vars='index', var_name='p')
data_viz['p'] = data_viz.p.astype('category')

sns.set_style('whitegrid')
lp = sns.relplot(data=data_viz, kind='line', x='index', y='value', hue='p')
lp.fig.set_figwidth(20)
lp.fig.set_figheight(6)
lp.fig.suptitle(f'Распространение инфекции с {airport}')
lp.ax.set_xlabel('Время')
lp.ax.set_ylabel('Средний процент инфицированных аэропортов')
plt.show()

# Может ли информация о сети помочь предсказать, кто будет заражен?

In [ ]:
@np.vectorize
def get_edge(origin, dest):
    """
    Функция для создания рёбер графа
    """
    if origin < dest:
        result = f'{origin} {dest}'
    else:
        result = f'{dest} {origin}'
    return result

Создадим граф

In [ ]:
# graph=nx.DiGraph()
# graph.add_weighted_edges_from(flights)
# pos = nx.spring_layout(graph, k=5)
# nx.draw(graph, pos, with_labels=True)

In [ ]:
# graph.number_of_nodes()

In [ ]:
# graph.number_of_edges()